In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('../datas/RGB_color_labels.csv') # read the dataset using pandas library
dataset = pd.get_dummies(dataset, columns=['label']) # adds 11 new columns that one hot encodes the class of the color
dataset


,red,green,blue,label_Black,label_Blue,label_Brown,label_Green,label_Grey,label_Orange,label_Pink,label_Purple,label_Red,label_White,label_Yellow
0,20,139,240,0,1,0,0,0,0,0,0,0,0,0
1,174,83,72,0,0,1,0,0,0,0,0,0,0,0
2,144,249,131,0,0,0,1,0,0,0,0,0,0,0
3,168,25,156,0,0,0,0,0,0,1,0,0,0,0
4,30,182,136,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047,26,26,26,1,0,0,0,0,0,0,0,0,0,0
5048,27,27,27,1,0,0,0,0,0,0,0,0,0,0
5049,28,28,28,1,0,0,0,0,0,0,0,0,0,0
5050,29,29,29,1,0,0,0,0,0,0,0,0,0,0


In [3]:
train_dataset = dataset.sample(frac=0.8, random_state=8) #train_dataset = 80% of total dataset  
#random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within 
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset

In [4]:
# Separates the dataset(that will be the input) and the labels(that will be the class of the input values). This part is for training.
train_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    train_labels[x] = train_dataset.pop(x)

In [5]:
# Separates the dataset(that will be the input) and the labels(that will be the class of the input values). This part is for testing.
test_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    test_labels[x] = test_dataset.pop(x)

In [6]:
# Converts all data into numpy arrays
# Why convert? Because the code gives a lot of errors if you did not input a numpy or tensors array to the neural network.
# I will leave to the reader to research about tensors in the documentation. https://www.tensorflow.org/guide/tensor
train_dataset = np.array(train_dataset)
train_labels = np.array(train_labels)
train_labels = np.argmax(train_labels, axis=1)

test_dataset = np.array(test_dataset)
test_labels = np.array(test_labels)
test_labels = np.argmax(test_labels, axis=1)


In [7]:
# Function for creating a deep neural network model
# 1 input layer - 3 neurons(RGB values)
# 4 hidden layers - 15 neurons each layer
# 1 output layer - 11 neurons(11 number of different classes ex. label_Black, label_Blue, etc)
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(3, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(3,)))
    for i in range(1,5):
        model.add(tf.keras.layers.Dense(15, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
    model.add(tf.keras.layers.Dense(11))

    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.build()

    return model

In [8]:
# Save the model in a variable
model = create_model()

In [9]:
# this saves the whole model including the loss function used, everything etc.
import os
from datetime import datetime

global models_path
models_path = "../trained_models"


if os.listdir(f"{models_path}"):
    # Makes the model folder
    last_model_num = os.listdir(f"{models_path}")[-1][-1]
    os.mkdir(f"{models_path}/model{int(last_model_num) + 1}")
    
    # Makes a readme file that contains information on when was this created
    last_model = os.listdir(f"{models_path}")[-1]
    with open(f"{models_path}/{last_model}/readme.txt", "w+") as f:
        f.write(f"Model created on: {str(datetime.now())}")

# If there are no models on path
elif not os.listdir(f"{models_path}"):
    # Makes the model folder
    os.mkdir(f"{models_path}/model1")

    # Makes a readme file that contains information on when was this created
    with open(f"{models_path}/model1/readme.txt", "w+") as f:
        f.write(f"Model created on: {str(datetime.now())}")

last_model = os.listdir(f"{models_path}")[-1]
os.makedirs(f"{models_path}/{last_model}/saved_per_train/train1")


In [10]:
from tensorflow.keras.callbacks import CSVLogger

checkpoint_path = f"{models_path}/{last_model}/saved_per_train/train1" + "/Epoch{epoch:02d}_loss{loss:.2f}"
csv_logger = CSVLogger(f"{models_path}/{last_model}/saved_per_train/train1/logs.csv", separator=',', append=False)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 monitor='accuracy',
                                                 save_freq=100) # if save_freq='epochs' it saves the model per epoch
                                                                # if save_freq=int_type it saves the model per <int_type> of batches
# Train the model with the new callback
model.fit(train_dataset, 
          train_labels,  
          epochs=100,
          batch_size = 1000,
          validation_data=(test_dataset, test_labels),
          callbacks=[cp_callback, csv_logger], # Pass callback to training
          shuffle=True)  

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/100
5/5 [==============================] - 3s 171ms/step - loss: 45.9254 - accuracy: 0.1168 - val_loss: 40.8680 - val_accuracy: 0.1059
Epoch 2/100
5/5 [==============================] - 0s 15ms/step - loss: 35.8766 - accuracy: 0.1168 - val_loss: 31.5798 - val_accuracy: 0.1059
Epoch 3/100
5/5 [==============================] - 0s 15ms/step - loss: 27.6602 - accuracy: 0.1163 - val_loss: 24.4178 - val_accuracy: 0.0921
Epoch 4/100
5/5 [==============================] - 0s 25ms/step - loss: 21.5108 - accuracy: 0.1059 - val_loss: 18.8906 - val_accuracy: 0.0891
Epoch 5/100
5/5 [==============================] - 0s 15ms/step - loss: 16.4910 - accuracy: 0.1071 - val_loss: 14.1482 - val_accuracy: 0.1020
Epoch 6/100
5/5 [==============================] - 0s 33ms/step - loss: 12.2469 - accuracy: 0.1165 - val_loss: 10.3389 - val_accuracy: 0.1059
Epoch 7/100
5/5 [==============================] - 0s 15ms/step - loss: 9.1303 - accuracy: 0.0789 - val_loss: 8.4571 - val_accuracy: 0.0406
Epoch 8

In [11]:
# This part finds the latest train in the model
last_trained = os.listdir(f"{models_path}/{last_model}/saved_per_train/")[-1]
last_trained_checkpoint_list = os.listdir(f"{models_path}/{last_model}/saved_per_train/{last_trained}")
last_trained_checkpoint_list.remove('logs.csv') # deletes the csv logs file in the list because we dont need it

In [12]:
# this part saves the weights in a excel file
import xlsxwriter

for model in last_trained_checkpoint_list:
    loaded_model = tf.keras.models.load_model(f"{models_path}/{last_model}/saved_per_train/{last_trained}/{model}")

    relative_row_idx = 0 # this is for writing bias
    row_idx = 0
    max_row_idx = 0
    column_idx = 0

    workbook = xlsxwriter.Workbook(f"{models_path}/{last_model}/saved_per_train/{last_trained}/{model}/saved_weights_biases.xlsx")
    worksheet = workbook.add_worksheet()

    for layer in loaded_model.layers:
        for row_weights in layer.get_weights()[0].T: # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
            for weights in row_weights:
                worksheet.write(row_idx, column_idx, weights)
                column_idx += 1

            column_idx += 1
            worksheet.write(row_idx, column_idx, layer.get_weights()[1].T[relative_row_idx]) # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
            relative_row_idx += 1
            row_idx += 1
            column_idx = 0

        relative_row_idx = 0
        row_idx += 2
    workbook.close()

In [13]:
# this part just evaluates the model how accurate it is in predicting data that it have not seen before
model.evaluate(test_dataset, test_labels, verbose=2)

AttributeError: 'str' object has no attribute 'evaluate'

In [ ]:
model.predict(train_dataset)[0]

127/127 [==============================] - 0s 1ms/step


array([ -5.307821  ,  -5.9415846 ,  -0.9156288 , -10.054424  ,
        -6.2752447 ,   3.4066713 ,   3.8608985 ,  -2.1819472 ,
         0.38721243, -13.704093  ,  -3.354857  ], dtype=float32)

In [ ]:
# this loads the whole model saved
# verbose just means what do you want to see in the outputs(what metrics do you want to see etc.)
new_model = tf.keras.models.load_model(f"{models_path}/{last_model}/saved_per_train/{last_trained}/{model}")
new_model.evaluate(test_dataset, test_labels, verbose=2)

32/32 - 0s - loss: 1.2891 - accuracy: 0.5574 - 257ms/epoch - 8ms/step


[1.2890599966049194, 0.5574257373809814]